##### 사용자 정의 함수

- - -
- 함수 기능 : 모델 학습 진행 함수
- 함수 이름 : training
- 함수 매개변수 : 함수 구동 시 필요한 재료 => 학습을 위한 재료
    * 모델인스턴스
    * 학습 데이터셋 : feature, target (학습 전 Tensor화) 
    * 손실 함수 인스턴스 : loss function
    * 최적화 인스턴스 : optimizer
    * 학습횟수 : 에포크 epoch
    * 배치 크기 : batch_size
    * 배치 개수 : batch_count
    * 검증용 데이터셋 : feature, target (검증 전 Tensor화)
- 함수 결과 : 학습 시 에포크 당 손실값과 성능지표값, 검증의 손실값과 성능지표값 반환
- - -

- - -
- 함수 기능 : 에포크 단위 모델 학습 진행 함수
- 함수 이름 : epochTraining
- 매개 변수 : 함수 구동 시 필요한 재료 => 학습을 위한 재료
    * 모델 인스턴스
    * 학습용 데이터셋 : feature, label(학습 전 Tensor화)
    * 손실 함수 인스턴스 : loss function
    * 최적화 인스턴스 : optimizer
    * 배치 크기 : batch_size
    * 배치 개수 : batch_count
- 함수 결과 : 손실값과 성능지표 값
- - -

In [2]:
## 모듈 로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optimizer
from torchmetrics.regression import R2Score
from torchmetrics.classification import F1Score

# 텐서 저장 및 실행 위치 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
## 에포크 당 학습 진행 후 손실값과 성능지표값 반환 함수
def epochTraining(model, feature, target, lossFunc, scoreFunc, optimizer, batch_cnt, batch_size=32, is_class=True):
    # 에포크에서 배치크기만큼 데이터셋 추출 후 학습 진행
    loss_total = 0
    score_total = 0
    for batch in range(batch_cnt):
        # 배치크기만큼 데이터셋 추출 및 텐서화
        start = batch*batch_size    # 예) 배치 사이즈가 32일 때 인덱스 0,32,64,96,...
        end = start+batch_size      # 예) 배치 사이즈가 32일 때 인덱스 32,64,96,...

        x_train = torch.FloatTensor(feature[start:end].values).to(DEVICE)
        y_train = torch.FloatTensor(target[start:end].values).to(DEVICE)

        # 학습 진행
        pre_y = model(x_train).to(DEVICE)
        
        # 손실 계산
        loss = lossFunc(pre_y, y_train)
        loss_total += loss
        
        # 점수 추출
        score = scoreFunc(pre_y, y_train)
        score_total += score

        # 최적화 : Weight, bias 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 에포크당 손실값과 성능지표값 반환
    # 테스트 및 성능 검증 함수 사용
    return loss_total/batch_cnt, score_total/batch_cnt

In [ ]:
## 검증 및 테스트 진행 후 손실값과 성능지표값 반환 함수
def testing(model, feature, target, lossFunc, scoreFunc):

    # 최적화 기능 비활성화 후 데이터셋 추출 후 학습 진행
    with torch.no_grad():
        # 데이터셋 텐서화
        featureTS = torch.FloatTensor(feature.values).to(DEVICE)
        targetTS = torch.FloatTensor(target.values).to(DEVICE)

        # 학습 진행
        pre_y = model(featureTS).to(DEVICE)
        
        # 손실 계산
        loss = lossFunc(pre_y, targetTS)
        
        # 점수 추출
        score = scoreFunc(pre_y, targetTS)

    # 에포크당 손실값과 성능지표값 반환
    # 테스트 및 성능 검증 함수 사용
    return loss, score

In [ ]:
# 학습 진행
EPOCHS = 10
Train_Validation_Loss = {'Train':[], 'Validation':[]}
Train_Validation_Score = {'Train':[], 'Validation':[]}
for epoch in (EPOCHS):
    # 학습 진행
    train_loss, trainscore = epochTraining()

    # 검증 진행
    val_loss, valscore = testing()

    # 에포크당 학습 및 검증 결과 저장
    Train_Validation_Loss['Train'].append(train_loss)
    Train_Validation_Score['Train'].append(trainscore)
    
    Train_Validation_Loss['Validation'].append(val_loss)
    Train_Validation_Score['Validation'].append(valscore)

    # 에포크당 학습 및 검증 결과 출력
    print(f"[{epoch}.{EPOCHS}]\n- Loss : {train_loss}, Score : {trainscore}")
    print(f"[{epoch}.{EPOCHS}]\n- Loss : {val_loss}, Score : {valscore}")